In [13]:
import requests
import folium
from IPython.display import display

# Function to decode the encoded points of a polygon
def decode_points(encoded):
    length = len(str(encoded))
    index = 0
    ar = []
    lat = 0
    lng = 0

    try:
        while index < length:
            # Decoding latitude
            b = 0
            shift = 0
            result = 0
            while True:
                # Getting the ASCII value of the character and subtracting 63
                # to get the original value of the encoded character
                b = ord(encoded[index]) - 63
                index += 1
                # Extracting 5 bits from the character and shifting them based on the position
                # to get the original value of the latitude
                result |= (b & 0x1f) << shift
                shift += 5
                # Checking if the next character is the continuation of the current value
                if b < 0x20:
                    break

            # Calculating the actual latitude from the decoded value
            dlat = ~result >> 1 if result & 1 else result >> 1
            lat += dlat

            # Decoding longitude (similar to decoding latitude)
            shift = 0
            result = 0
            while True:
                b = ord(encoded[index]) - 63
                index += 1
                result |= (b & 0x1f) << shift
                shift += 5
                if b < 0x20:
                    break

            dlng = ~result >> 1 if result & 1 else result >> 1
            lng += dlng

            # Adding the decoded coordinates to the list
            ar.append((lat * 1e-5, lng * 1e-5))
    
    except Exception as ex:
        # Error in encoding, handle or ignore
        pass

    return ar

# API endpoints for the GeoJSON and data
geo_endpoint = 'http://geo-exercise.id.com.au/api/geo'
data_endpoint = 'http://geo-exercise.id.com.au/api/data'

# Fetching the GeoJSON and data from the API endpoints
geo_response = requests.get(geo_endpoint)
data_response = requests.get(data_endpoint)

# Parsing the response as JSON
geo_data = geo_response.json()
data = data_response.json()

# Decoding the polygon shapes
decoded_polygons = []
for shape in geo_data['shapes']:
    polygon_id = shape['id']
    points = decode_points(shape['points'])
    decoded_polygons.append((polygon_id, points))

# Extracting the map center from the first polygon's coordinates
map_center = (decoded_polygons[0][1][0][1], decoded_polygons[0][1][0][0])  # Fix the order of coordinates for map center
map_zoom = 10

# Creating a Leaflet map object
m = folium.Map(location=map_center, zoom_start=map_zoom)

# Adding polygons to the map
for polygon_id, points in decoded_polygons:
    polygon_color = None
    
    # Finding the matching color for the polygon from the data
    for entry in data['data']:
        if entry['GeoID'] == polygon_id:
            polygon_color = entry['color']
            break
    
    # Fixing the order of coordinates and creating a Polygon object for each polygon
    coordinates = [(point[1], point[0]) for point in points]  # Fix the order of coordinates for each point
    
    folium.Polygon(
        locations=coordinates,
        fill_color=polygon_color,
        fill_opacity= 0.8,
        color='cB',
        weight=7
    ).add_to(m)

# Displaying the map in the Jupyter Notebook
display(m)
